<a href="https://colab.research.google.com/github/shreeshaa/BDA-homework-1/blob/master/WebSearch3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tweet-preprocessor
import pandas as pd
import numpy as np
import scipy as sp
import re
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from copy import deepcopy
from bs4 import BeautifulSoup
import nltk
nltk.download('stopwords')
from nltk.stem import PorterStemmer 

import preprocessor as p

  Created wheel for tweet-preprocessor: filename=tweet_preprocessor-0.5.0-cp36-none-any.whl size=7947 sha256=693412f6aed5a2e70c62daebafd9349080b1d7a9ec9aae46983d9dac2a7ca108
  Stored in directory: /root/.cache/pip/wheels/1b/27/cc/49938e98a2470802ebdefae9d2b3f524768e970c1ebbe2dc4a
Successfully built tweet-preprocessor
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
train = pd.read_csv("/content/drive/My Drive/WebProject/labeledTrainData.csv", delimiter="\t",names=["tweet", "label", "labelValue"])

In [0]:
train["label"].replace({"spam": "neutral", "normal": "neutral"}, inplace=True)

In [5]:
train.head()


,tweet,label,labelValue
0,Beats by Dr. Dre urBeats Wired In-Ear Headphon...,neutral,4
1,RT @Papapishu: Man it would fucking rule if we...,abusive,4
2,It is time to draw close to Him &#128591;&#127...,neutral,4
3,if you notice me start to act different or dis...,neutral,5
4,"Forget unfollowers, I believe in growing. 7 ne...",neutral,3


In [0]:
def expand_hashtags(tweet):
    ws.load()
    new_tweet = "" # output    
    tokens = tweet.split(' ')    
    for t in tokens:
        if t[0] == '#': # this is a hashtag, parse it
            hashtags_tokens = ws.segment(t)
            exp = " ".join(hashtags_tokens)
            new_tweet += exp
        else: # Just append the word
            new_tweet += t
        new_tweet += " "

    return new_tweet 


In [0]:
####Preprocessing for K_Means

def preprocess_tweet_KMeans(tweet):

    # 1. Remove URLs, emojis, mentions, smileys using tweet-preprocessor library
    # should not remove hashtags as hashtags contains important tweet content.
    
    #p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION,p.OPT.RESERVED,	p.OPT.SMILEY,p.OPT.NUMBER)
    clean = p.clean(tweet)
    # 2. Remove HTML tags using Beautiful soup library
    no_tag = BeautifulSoup(clean).get_text()

    # 3. Expanding hashtags.
    
    #tweets_expanded = expand_hashtags(no_tag)

    # 4. Remove non letter char using re
    letters_only = re.sub("[^a-zA-Z]"," ", no_tag)
    #letters_only = re.sub("[^a-zA-Z]"," ", tweets_expanded)
    
    # 5. Convert to lower case
    lower_case = letters_only.lower()
    words = lower_case.split()

    new_words = []
    # ps = PorterStemmer() 
    for w in words:
        new_words.append(w)
    # print(new_words[0:4])
    # 5. Remove stop words
    stops = set(stopwords.words("english")) 
    final_words = np.array([w for w in new_words if not w in stops])

    return final_words

In [36]:
cleaned_tweet_KMeans = np.array([])
labelss = np.array([])

num_tweets = train["tweet"].size
# print(num_tweets)
for i in range(num_tweets):
    if len(preprocess_tweet_KMeans(train["tweet"][i]))>0:
      if train["label"][i] == 'abusive':
        labelss = np.concatenate((labelss,np.array([0])),axis=0)
        # labelss.append(0)
        cleaned_tweet_KMeans = np.concatenate((cleaned_tweet_KMeans, preprocess_tweet_KMeans(train["tweet"][i])), axis=0)
        # cleaned_tweet_KMeans.append(preprocess_tweet_KMeans(train["tweet"][i]))
      elif train["label"][i] == 'hateful':
        labelss = np.concatenate((labelss,np.array([1])),axis=0)
        cleaned_tweet_KMeans = np.concatenate((cleaned_tweet_KMeans, preprocess_tweet_KMeans(train["tweet"][i])), axis=0)
        # labelss.append(1)
        # cleaned_tweet_KMeans.append(preprocess_tweet_KMeans(train["tweet"][i]))
      else:
        labelss = np.concatenate((labelss,np.array([2])),axis=0)
        cleaned_tweet_KMeans = np.concatenate((cleaned_tweet_KMeans, preprocess_tweet_KMeans(train["tweet"][i])), axis=0)
        # cleaned_tweet_KMeans.append(preprocess_tweet_KMeans(train["tweet"][i]))
        # labelss.append(2)

KeyboardInterrupt: ignored

In [37]:
print(cleaned_tweet_KMeans[0])

beats


In [28]:
cleaned_tweet_KMeans = np.vstack(cleaned_tweet_KMeans)
print(cleaned_tweet_KMeans[0])
# cleaned_tweet_KMeans_top100 = cleaned_tweet_KMeans[:1000]
to_consider = cleaned_tweet_KMeans
# print(cleaned_tweet_KMeans[319])


ValueError: ignored

In [0]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
%matplotlib inline

import seaborn as sns
sns.set_style('darkgrid')
sns.set_palette('muted')
sns.set_context("notebook", font_scale=1.5,
                rc={"lines.linewidth": 2.5})
RS = 123

In [0]:
def fashion_scatter(x, colors):
    # choose a color palette with seaborn.
    num_classes = len(np.unique(colors))
    palette = np.array(sns.color_palette("hls", num_classes))

    # create a scatter plot.
    f = plt.figure(figsize=(8, 8))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:,0], x[:,1], lw=0, s=40, c=palette[colors.astype(np.int)])
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)
    ax.axis('off')
    ax.axis('tight')

    # add the labels for each digit corresponding to the label
    txts = []
    for i in range(num_classes):

        # Position of each label at median of data points.

        xtext, ytext = np.median(x[colors == i, :], axis=0)
        txt = ax.text(xtext, ytext, str(i), fontsize=24)
        txt.set_path_effects([
            PathEffects.Stroke(linewidth=5, foreground="w"),
            PathEffects.Normal()])
        txts.append(txt)

    return f, ax, sc, txts

In [0]:
from sklearn.decomposition import PCA

time_start = time.time()

pca_50 = PCA(n_components=50)
print(to_consider.shape)
pca_result_50 = pca_50.fit_transform(to_consider)

print (time.time()-time_start)

print (np.sum(pca_50.explained_variance_ratio_))